In [4]:
!export LC_ALL=en_US.UTF-8
import tensorflow as tf
import json
from pprint import pprint
import re
import numpy as np
from embedding import *

In [6]:
data = json.load(open('train-v1.1.json'))

In [7]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [8]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [10]:
X_train_comp_all = []
X_train_comp_ans_all = []
X_train_ans_all = []
Y_train_ques_all = []
invalid = 0
X_train_ans_label_all = []
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            continue
        a_lab[start:end+1] = 1
            
            
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_ans_all.append(findAnsVec(answer,passage))
            X_train_ans_label_all.append(a_lab)
            X_train_comp_all.append(cappedPassage)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
    

In [11]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all for item in sublist] + [item for sublist in Y_train_ques_all for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [12]:
wordToTake = 1000
words = findKMostFrequentWords(50000)

In [13]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

(1000, 100)

In [14]:
!hostname
print(invalid)
for i in np.where(X_train_ans_label_all[110] == 1)[0]:
    print(X_train_comp_all[110][i])

JackSparrow.fios-router.home
1493
rev
.
john
j.
cavanaugh
,
c.s.c
.
served
more
than
half
,
lobund
institute
for
animal
studies
and
medieval
institute
.
hall
of
liberal
arts
(


In [15]:
print(X_train_comp_all[0])
print(X_train_ans_all[0])

['architecturally', ',', 'the', 'school', 'has', 'a', 'catholic', 'character', '.', 'atop', 'the', 'main', 'building', "'s", 'gold', 'dome', 'is', 'a', 'golden', 'statue', 'of', 'the', 'virgin', 'mary', '.', 'immediately', 'in', 'front', 'of', 'the', 'main', 'building', 'and', 'facing', 'it', ',', 'is', 'a', 'copper', 'statue', 'of', 'christ', 'with', 'arms', 'upraised', 'with', 'the', 'legend', '``', 'venite', 'ad', 'me', 'omnes', "''", '.', 'next', 'to', 'the', 'main', 'building', 'is', 'the', 'basilica', 'of', 'the', 'sacred', 'heart', '.', 'immediately', 'behind', 'the', 'basilica', 'is', 'the', 'grotto', ',', 'a', 'marian', 'place', 'of', 'prayer', 'and', 'reflection', '.', 'it', 'is', 'a', 'replica', 'of', 'the', 'grotto', 'at', 'lourdes', ',', 'france', 'where', 'the', 'virgin', 'mary', 'reputedly', 'appeared', 'to', 'saint', 'bernadette', 'soubirous', 'in', '1858.', 'at', 'the', 'end', 'of', 'the', 'main', 'drive', '(', 'and', 'in', 'a', 'direct', 'line', 'that', 'connects', 't

In [16]:
find_sub_list(X_train_ans_all[0] , X_train_comp_all[0])

(102, 105)

In [17]:
print(invalid)
print(X_train_comp_all[101])
print(X_train_ans_all[101])
print(Y_train_ques_all[101])

c = list(zip(X_train_comp_all,X_train_comp_ans_all, X_train_ans_all, X_train_ans_label_all,Y_train_ques_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_comp_ans_all_shuffled, X_train_ans_shuffled, X_train_ans_label_shuffled,Y_train_ques_all_shuffled = zip(*c)

print(X_train_comp_all_shuffled[101])
print(X_train_comp_ans_all_shuffled[101])
print(X_train_ans_shuffled[101])
print(X_train_ans_label_shuffled[101])
print(Y_train_ques_all_shuffled[101])


1493
['the', 'success', 'of', 'its', 'football', 'team', 'made', 'notre', 'dame', 'a', 'household', 'name', '.', 'the', 'success', 'of', 'note', 'dame', 'reflected', 'rising', 'status', 'of', 'irish', 'americans', 'and', 'catholics', 'in', 'the', '1920s', '.', 'catholics', 'rallied', 'up', 'around', 'the', 'team', 'and', 'listen', 'to', 'the', 'games', 'on', 'the', 'radio', ',', 'especially', 'when', 'it', 'knocked', 'off', 'the', 'schools', 'that', 'symbolized', 'the', 'protestant', 'establishment', 'in', 'america', '—', 'harvard', ',', 'yale', ',', 'princeton', ',', 'and', 'army', '.', 'yet', 'this', 'role', 'as', 'high-profile', 'flagship', 'institution', 'of', 'catholicism', 'made', 'it', 'an', 'easy', 'target', 'of', 'anti-catholicism', '.', 'the', 'most', 'remarkable', 'episode', 'of', 'violence', 'was', 'the', 'clash', 'between', 'notre', 'dame', 'students', 'and', 'the', 'ku', 'klux', 'klan', 'in', '1924.', 'nativism', 'and', 'anti-catholicism', ',', 'especially', 'when', 'dire

In [18]:
examples_to_take_train = 10

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_comp_ans = X_train_comp_ans_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_shuffled[0:examples_to_take_train]
X_train_ans_label = X_train_ans_label_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
answer_indices = [np.where(x==1)[0].tolist() for x in X_train_comp_ans]

In [19]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [20]:
document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
suppression_answer = np.zeros((examples_to_take_train, reduced_glove.shape[0], 1),dtype=np.int32)
print(suppression_answer.shape)

(10, 1000, 1)


In [21]:
print(answer_labels.shape)
for i in range(examples_to_take_train):
    answer_labels[i,0:len(X_train_ans_label[i])] = X_train_ans_label[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    for j, index in enumerate(answer_indices[i]):
        answer_masks[i, j, index] = 1
    answer_lengths[i] = len(answer_indices[i])
    
    #print(Y_train_ques[i])
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)
    
    for j, word in enumerate(X_train_ans[i]):
        if(word not in Y_train_ques[i]):
            suppression_answer[i, look_up_word_reduced(word),:] = 1

(10, 181)


In [22]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [23]:
print(reduced_glove.shape)
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

(1000, 100)
569
48
545


In [24]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [25]:
def sentences_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
    return X_indices

In [26]:
document_tokens = sentences_to_indices_glove(X_train_comp, max_document_len)

In [27]:
document_tokens[0]

array([  3,   0,   5, 475,   0,   5,   3, 346,   5, 553,   0,  24,   0,
         0,  25,  20,   3, 209,   5,   3,  24,   0, 248, 306, 173, 231,
        25,  51,  38,   0, 114, 348, 173, 231,   6,   3,   0,   5, 475,
         0,   5,   3, 346,   5,   0,  40, 141,  41,  94,   3, 241, 348,
       173,   6,  42, 648, 163,   3, 306, 173,   4,   3,   0,   7, 314,
         6,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   

In [28]:
answer_labels.shape

(10, 181)

## Model

### Document Embedding

In [29]:
import tensorflow.contrib.seq2seq as seq2seq

tf.reset_default_graph()

embedding = tf.get_variable("embedding", initializer=reduced_glove)
embedding = tf.cast(embedding, dtype=tf.float32)
EMBEDDING_DIMENS = glove.shape[1]

d_tokens = tf.placeholder(tf.int32, shape=[None, None], name="d_tokens")
d_lengths = tf.placeholder(tf.int32, shape=[None], name="d_lengths")


document_emb = tf.nn.embedding_lookup(embedding, d_tokens, name="document_emb")
document_emb = tf.cast(document_emb, dtype=tf.float64, name="casted_document_emb")


Instructions for updating:
Use the retry module or similar alternatives.


### Answer Generation

In [30]:

forward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)
backward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)

answer_outputs, _ = tf.nn.bidirectional_dynamic_rnn(
    forward_cell, backward_cell, document_emb, d_lengths, dtype=tf.float64,
    scope="answer_rnn")

answer_outputs = tf.concat(answer_outputs, 2, name="answer_output_concat")

answer_outputs = tf.cast(answer_outputs,tf.float32, name="answer_output_concat")

answer_tags = tf.layers.dense(inputs=answer_outputs, units=2, name="answer_tags")


a_labels = tf.placeholder(tf.int32, shape=[None, None], name="a_labels")

answer_mask = tf.sequence_mask(d_lengths, dtype=tf.float32, name="answer_mask")

answer_loss = seq2seq.sequence_loss(
    logits=answer_tags, targets=a_labels, weights=answer_mask, name="answer_loss")

## Question Generation

### Encoder

In [31]:
encoder_input_mask = tf.placeholder(
    tf.float32, shape=[None, None, None], name="encoder_input_mask")
encoder_inputs = tf.matmul(encoder_input_mask, answer_outputs, name="encoder_inputs")
encoder_lengths = tf.placeholder(tf.int32, shape=[None], name="encoder_lengths")

encoder_cell = tf.contrib.rnn.GRUCell(forward_cell.state_size + backward_cell.state_size)

_, encoder_state = tf.nn.dynamic_rnn(
    encoder_cell, encoder_inputs, encoder_lengths, dtype=tf.float32, scope="encoder_rnn")

### Decoder

In [32]:
from tensorflow.python.layers.core import Dense


decoder_inputs = tf.placeholder(tf.int32, shape=[None, None], name="decoder_inputs")
decoder_lengths = tf.placeholder(tf.int32, shape=[None], name="decoder_lengths")

decoder_emb = tf.nn.embedding_lookup(embedding, decoder_inputs,name="decoder_embedding")
decoder_emb = tf.cast(decoder_emb,tf.float32,name="decoder_embedding_cast")

#helper = seq2seq.TrainingHelper(decoder_emb , decoder_lengths2, name="helper")
helper = seq2seq.TrainingHelper(decoder_emb , decoder_lengths, name="helper")


projection = Dense(embedding.shape[0], use_bias=False, name="projection")

decoder_cell = tf.contrib.rnn.GRUCell(encoder_cell.state_size)


decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, scope="decoder")
decoder_outputs = decoder_outputs.rnn_output

decoder_labels = tf.placeholder(tf.int64, shape=[None, None], name="decoder_labels")
question_mask = tf.sequence_mask(decoder_lengths ,dtype=tf.float32)




## Question Generation Loss

In [33]:
# NLL Loss
question_loss = seq2seq.sequence_loss(
    logits=decoder_outputs, targets=decoder_labels, weights=question_mask,
    name="question_loss")


#Suppression Loss
s_answer = tf.placeholder(tf.float32, shape=[None,None,None], name="suppression_answer")
lambdaSuppress = 0.5
decoder_outputs = tf.Print(decoder_outputs, [decoder_outputs], message="This is decoder_outputs: ")
suppression_loss = lambdaSuppress * tf.reduce_sum(tf.matmul(tf.exp(decoder_outputs), s_answer))
suppression_loss = tf.Print(suppression_loss, [suppression_loss], message="This is suppression_loss: ")

In [34]:
import spacy
nlp = spacy.load('en_core_web_lg')

ModuleNotFoundError: No module named 'spacy'

In [142]:
q1 = np.array[nlp(' '.join(['what','is','up'])), nlp(' '.join(['what','is','up']))]
q2 = an.array[nlp(' '.join(['how','are','you'])), nlp(' '.join(['how','are','you']))]
q1.similarity(q2)

TypeError: 'builtin_function_or_method' object is not subscriptable

In [144]:
q1 = nlp(' '.join(['what','is','up']))
q2 = nlp(' '.join(['what','is','down']))
q1.similarity(q2)


0.97599522047520637

In [132]:
decoder_labels[0]

<tf.Tensor 'strided_slice:0' shape=(?,) dtype=int64>

In [88]:
look_up_token_reduced(26)

'what'

In [96]:
look_up_word_reduced('you')

746

In [139]:
def similarity_score(decoder_outputs, decoder_labels,question_mask):
    #decoder_outputs = np.argmax(decoder_outputs,axis=2)
    #output_tokens = np.array(list(map(look_up_token_reduced, decoder_outputs)))
    #label_tokens = np.array(map(look_up_token_reduced,decoder_labels))
    #print(output_tokens)

    #output_tokens_masked = np.multiply(output_tokens,question_mask)
    #label_tokens_masked = np.multiply(label_tokens,question_mask)
    
    #output = np.apply_along_axis(' '.join(), 0, output_tokens_masked)
    #labels = np.apply_along_axis(' '.join(), 0, label_tokens_masked)

    #similarity_score = 
    output_tokens = []
    for i in range(decoder_outputs.shape[0]):
        temp = []
        for j in range(decoder_outputs.shape[1]):
            if(question_mask[i][j] == 0):
                break
            temp.append(look_up_token_reduced(decoder_outputs[i][j]))
            #output_tokens[i][j] = look_up_token_reduced(decoder_outputs[i][j])
        output_tokens.append(temp)
    label_tokens = []
    for i in range(decoder_labels.shape[0]):
        temp = []
        for j in range(decoder_labels.shape[1]):
            if(question_mask[i][j] == 0):
                break
            temp.append(look_up_token_reduced(decoder_labels[i][j]))
        label_tokens.append(temp)
    
    print(output_tokens)
    print(label_tokens)
    similarity_score = np.zeros(decoder_outputs.shape[0])
    for i in range(similarity_score.shape[0]):
        q1 = nlp(' '.join(output_tokens[i]))
        q2 = nlp(' '.join(label_tokens[i]))
        similarity_score[i] = q1.similarity(q2)
    return(similarity_score)
    

In [140]:
my_outputs = np.array([[26,11,118],[89,27,746]])
my_labels = np.array([[26,11,388],[26,11,388]])
my_mask = np.array([[1,1,1,0,0],[1,1,1,0,0]])
score = similarity_score(my_outputs,my_labels,my_mask)

[['what', 'is', 'up'], ['how', 'are', 'you']]
[['what', 'is', 'down'], ['what', 'is', 'down']]


In [141]:
score

array([ 0.97599522,  0.80616866])

In [ ]:
#decoder_outputs_max = tf.argmax(decoder_outputs,axis=2)


In [61]:
look_up_token_reduced(0)

'<UNK>'

In [53]:
question_mask

(10,)

In [42]:
loss = tf.add(answer_loss, question_loss, name="loss")
loss = tf.add(loss, suppression_loss, name="loss")

In [43]:
#document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
#answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
#question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
#question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#suppression_answer = np.zeros((examples_to_take_train,max_answer_len),dtype=np.int32)

In [44]:
def shuffle_list(*ls):
    l =list(zip(*ls))
    np.random.shuffle(l)
    return zip(*l)

In [45]:
import math
def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    num_batches = math.ceil(len(inputs[0])/batch_size)
    
    for index,inp in enumerate(inputs):
        start = 0
        output = []
        for i in range(num_batches-1):
            maxD = max(inputs[1][start:start+batch_size])
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max(inputs[7][start:start+batch_size])
            if index == 0 or index == 2:
                output.append(inp[start:start+batch_size,0:maxD]) 
            elif index==3:
                output.append(inp[start:start+batch_size,0:maxA,0:maxD]) 
            elif index==5 or index==6:
                output.append(inp[start:start+batch_size,0:maxQ])
            elif index == 8: # 
                output.append(inp[start:start+batch_size,:,:]) 
            else: 
                output.append(inp[start:start+batch_size])
            start = start + batch_size
        
        # Remaining training sample i.e. training mod batch_size
        maxD = max(inputs[1][start:])
        maxA = max(inputs[4][start:])
        maxQ = max(inputs[7][start:])
        if index == 0 or index == 2:
            output.append(inp[start:,0:maxD]) 
        elif index==3:
            output.append(inp[start:,0:maxA,0:maxD]) 
        elif index==5 or index==6:
            output.append(inp[start:,0:maxQ]) 
        elif index ==8:
            output.append(inp[start:,:, :]) 
        else: 
            output.append(inp[start:])
        outputs.append(output)
    
    return outputs


In [46]:
batch_size = 10
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_masks,answer_lengths,question_input_tokens,question_output_tokens,question_lengths,suppression_answer]
                    ,batch_size)

In [47]:
"No of batches:",len( batch_input[0])

('No of batches:', 1)

In [48]:
optimizer = tf.train.AdamOptimizer(learning_rate=3e-3).minimize(loss)

saver = tf.train.Saver()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.InteractiveSession(config=config)

session.run(tf.global_variables_initializer())
# session.run(tf.variables_initializer(saved_vars))


EPOCHS = 20

for epoch in range(1, EPOCHS + 1):
    print("Epoch {0}".format(epoch))
    for batchNum in range(len(batch_input[0])):
        print("Batch : ",batchNum)
        t = session.run([optimizer, loss, decoder_outputs, suppression_loss], {
            d_tokens: batch_input[0][batchNum],
            d_lengths: batch_input[1][batchNum],
            a_labels: batch_input[2][batchNum],
            encoder_input_mask: batch_input[3][batchNum],
            encoder_lengths: batch_input[4][batchNum],
            decoder_inputs: batch_input[5][batchNum],
            decoder_labels: batch_input[6][batchNum],
            decoder_lengths: batch_input[7][batchNum],
            s_answer: batch_input[8][batchNum],
        })
        tf.Session().run()
        print("Loss: {0}".format(t[1]))
    if(epoch%5 == 0):
        print("Saving model")
        #saver.save(session, "qgen-model")


Epoch 1
Batch :  0


KeyboardInterrupt: 

In [39]:
print(t[2])

[[[-1.50505602 -3.07428956  2.56434155 ..., -3.07349539 -3.09046841
   -3.03399873]
  [-1.76248872 -3.63838267  3.2439065  ..., -3.60971045 -3.58260107
   -3.6037097 ]
  [-1.89708757 -3.88434219  3.52075863 ..., -3.82470012 -3.79168701
   -3.84030294]
  ..., 
  [-2.04667497 -4.1406498   3.82709146 ..., -4.07375765 -4.03911209
   -4.09230137]
  [-2.04751468 -4.14365482  3.82962894 ..., -4.07715559 -4.04227924
   -4.09470797]
  [-2.04816628 -4.14639616  3.83192492 ..., -4.08027935 -4.04514122
   -4.09690571]]

 [[-1.8915031  -3.94711781  3.36775589 ..., -3.78215742 -3.83540106
   -3.81771779]
  [-1.97481418 -4.14009571  3.65373945 ..., -3.9985826  -4.01781034
   -4.04910374]
  [-2.0104506  -4.19613743  3.75386572 ..., -4.06355572 -4.07419825
   -4.12057781]
  ..., 
  [-2.06712484 -4.23035145  3.86246133 ..., -4.13321733 -4.09855413
   -4.1683917 ]
  [-2.06826019 -4.22900057  3.86204839 ..., -4.13337708 -4.09799767
   -4.16759014]
  [-2.06893539 -4.2278266   3.86186838 ..., -4.13366032 -4

In [ ]:
saver = tf.train.Saver()
saver.save(session, "qgen-model")

In [ ]:
batch_input[3][0].shape

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
saver = tf.train.Saver()

saver.restore(session, 'qgen-model')

In [ ]:
answers = session.run(answer_tags, {
    d_tokens: batch_input[0][48],
    d_lengths: batch_input[1][48],
})
answers = np.argmax(answers, 2)

In [ ]:
for i in range(28):
    print("Prediction")
    printAllAns(answers,48,0)
    print("Ground Truth")
    printAllAns(batch_input[2][48],48,0)

In [ ]:
def printDoc(batch,num):
    for i in batch_input[0][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")

def printQues(batch,num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAnsForQuestion(batch, num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAllAns(answers, batch, num):
    for i,word in enumerate(batch_input[0][batch][num]):
        if answers[num][i] == 1 :
            print(look_up_token_reduced(word),sep=" ", end=" ")
    print(" ")
    
printQues(47,0)
    

In [ ]:
import itertools

batchNum = 80

helper = seq2seq.GreedyEmbeddingHelper(embedding, tf.fill([batch_input[0][batchNum].shape[0]], START_TOKEN), END_TOKEN)
decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, maximum_iterations=max_question_len)
decoder_outputs = decoder_outputs.rnn_output


questions = session.run(decoder_outputs, {
    d_tokens: batch_input[0][batchNum],
    d_lengths: batch_input[1][batchNum],
    a_labels: batch_input[2][batchNum],
    encoder_input_mask: batch_input[3][batchNum],
    encoder_lengths: batch_input[4][batchNum],
})



In [ ]:
batch_input[0][batchNum].shape

In [ ]:
#questions[:,:,END_TOKEN] = 0
qs = np.argmax(questions, 2)

In [ ]:
# q = set()
l = []
for i in range(batch_input[0][batchNum].shape[0]):
    print("---------------------------------------------------------------------------------------------")
    question = itertools.takewhile(lambda t: t != END_TOKEN, qs[i])
    print("Generated Question: " + " ".join(look_up_token_reduced(token) for token in question))
    print("Ground Truth Question: ")
    printQues(batchNum,i)
    print("Ground Truth Answer: ", X_train_ans_shuffled[batch_size*batchNum + i])
    print("Context:")
    printDoc(batchNum,i)
    print("---------------------------------------------------------------------------------------------")
    
    
# q

In [ ]:
batch_input[5][18].shape

In [ ]:
batch_input[5][1].shape

In [ ]:
batch_input[6][1].shape

In [ ]:
max(batch_input[7][1])

In [ ]:
max_question_len

In [ ]:
len(batch_input)

In [ ]:
reduced_glove